# Raw data extraction

In [5]:
source(here::here("scripts/init.R"))

Loading methylayer



## Extract raw methylation data

Due to the large size of the METABRIC methylation data we have to parallelize heavily. We do so using a custom api for a Sun Grid Engine (SGE) cluster, see `misha.ext::gcluster.run2` for details.

### Msp1 fragments

We average all CpGs on one Msp1 fragment. \
Fragments with less than 30 methylation calls are considered missing data (NA):

In [2]:
msp1_meth <- get_msp1_meth(min_cov = 30)

In [3]:
dim(msp1_meth$avg)
rm(msp1_meth)
gc()

[1] 2297198    2016

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2917104,155.8,4953054,264.6,4953054,264.6
Vcells,3503070886,26726.4,9823038413,74943.9,10448810008,79718.1


### Promoters

Promoters were defined as 500bp upstream and 50bp downstream from a RefSeq TSS (release 69, hg19). \
We extract average methylation of all CpGs covered in the promoter region. 

In [4]:
prom_meth <- get_promoter_meth(min_cov = 30)

In [5]:
dim(prom_meth$avg)
rm(prom_meth)
gc()

[1] 33241  2016

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2920980,156.0,4953054,264.6,4953054,264.6
Vcells,3583691734,27341.4,9823038413,74943.9,10448810008,79718.1


> Note that the above table contains multiple rows for the same promoter when alternative promoters exists.

#### Filter promoters

We keep only promoters that were 70% of the tumor samples and 70% of the normal samples. 

In [6]:
prom_avg_meth <- get_promoter_avg_meth(normal_fraction = 0.7, tumor_fraction = 0.7)

In [7]:
dim(prom_avg_meth)

[1] 15821  2016

> Note: this table contains only one row per genomic coordinates

### Non-promoter methylation ("Genomic")

We define a subset of Msp1 fragments that do not have a partial overlap to a defined promoter region or to an exon (within 10 bp of an exon). \
We keep only fragments that were 70% of the tumor samples and 70% of the normal samples. 

In [8]:
genomic_meth <- get_genomic_avg_meth(normal_fraction = 0.7, tumor_fraction = 0.7)
dim(genomic_meth)

[1] 193760   2016

### All average methylation loci

In order to get average methylation in all the loci, we concatenate the promoter and non-promoter methylation:

In [9]:
all_meth <- get_all_meth()
dim(all_meth)

[1] 209581   2016

### Exons

We compute separately the methylation on exons.

In [3]:
exon_meth <- get_exon_meth(min_cov = 30)

In [4]:
dim(exon_meth$avg)
rm(exon_meth)
gc()

[1] 244639   1785

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,4190375,223.8,7625532,407.3,5907582,315.5
Vcells,307423446,2345.5,973967811,7430.8,962491469,7343.3


#### Filter exons

We keep only exons that were 70% of the tumor samples and 70% of the normal samples. 

In [6]:
exon_avg_meth <- get_exon_avg_meth(normal_fraction = 0.7, tumor_fraction = 0.7)

In [7]:
dim(exon_avg_meth)

[1] 29512  1785

## Raw expression data

We match promoter methylation and gene expression profiles using Refseq annotations. \
Alternative promoters are resolved by selecting the promoter with the minimal average methylation value in the normal samples.

In [10]:
expr_mat <- get_gene_expression_mat()
expr_mat[1:5, 1:10]
dim(expr_mat)

,chrom,start,end,name,name3.chr,MB_0362,MB_0346,MB_0386,MB_0574,MB_0185
,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr19,58864814,58865365,A1BG,chr19.-.A1BG,5.607785,5.515704,5.581684,5.602209,5.560794
2,chr19,58862834,58863385,A1BG-AS1,chr19.+.A1BG-AS1,NA,NA,NA,NA,NA
3,chr10,52645384,52645935,A1CF,chr10.-.A1CF,5.409415,5.652667,5.364459,5.378185,5.485144
4,chr12,9268507,9269058,A2M,chr12.-.A2M,7.653319,6.564312,8.201633,7.587296,8.427318
5,chr12,9217271,9217822,A2M-AS1,chr12.+.A2M-AS1,NA,NA,NA,NA,NA


[1] 26245  2129

## Methylation per CpG

### Promoters

In [4]:
prom_cpg_meth <- get_promoter_cpgs_meth(min_cov = 1)

In [5]:
dim(prom_cpg_meth$avg)
rm(prom_cpg_meth)
gc()

[1] 33241  2016

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2920980,156.0,4953054,264.6,4953054,264.6
Vcells,3583691734,27341.4,9823038413,74943.9,10448810008,79718.1


In [6]:
prom_avg_meth_cpgs <- get_promoter_cpgs_avg_meth(normal_fraction = 0.7, tumor_fraction = 0.7)

In [7]:
dim(prom_avg_meth_cpgs)

[1] 15821  2016

### Non-promoters

In [8]:
genomic_meth_cpgs <- get_genomic_cpgs_avg_meth(normal_fraction = 0.7, tumor_fraction = 0.7)
dim(genomic_meth_cpgs)

[1] 193760   2016

In [ ]:
gc()